# 🚀 GPT-4-ENEM no Google Colab

Este notebook configura o ambiente para avaliar modelos de linguagem no ENEM usando GPUs do Google Colab (A100, T4, V100, etc.)

## 📋 Pré-requisitos
- Conta Google (para acessar Colab)
- Chave API da Maritaca (ou OpenAI)
- Acesso a GPU (gratuito ou Colab Pro)


## 1️⃣ Configurar GPU

**IMPORTANTE**: Vá em **Runtime → Change runtime type → Hardware accelerator → GPU**

Para A100: Requer **Colab Pro** ou **Colab Pro+**


In [ ]:
# Verificar GPU disponível
import torch

if torch.cuda.is_available():
    print(f"✅ GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"   Memória total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   CUDA version: {torch.version.cuda}")
else:
    print("⚠️  GPU não detectada. Certifique-se de ativar GPU em Runtime → Change runtime type")


## 2️⃣ Instalar Dependências


In [ ]:
# Instalar dependências principais
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers datasets scikit-learn
!pip install -q sqlitedict pytablewriter sacrebleu rouge-score
!pip install -q pycountry numexpr tqdm jsonlines
!pip install -q openai fschat
!pip install -q git+https://github.com/lm-sys/FastChat.git


In [ ]:
# Clonar repositório (ou fazer upload manual)
import os

# Opção 1: Clonar do GitHub
!git clone https://github.com/piresramon/gpt-4-enem.git
os.chdir('gpt-4-enem')

# Opção 2: Se já tiver os arquivos, fazer upload manual via Colab
# Files → Upload → selecionar arquivos do projeto

print("✅ Repositório clonado/configurado")


In [ ]:
# Instalar projeto em modo desenvolvimento
!pip install -e .

print("✅ Projeto instalado")


## 3️⃣ Configurar Chaves API


In [ ]:
import os

# Configurar chave API da Maritaca
# SUBSTITUA pela sua chave real
os.environ['CURSORMINIMAC'] = '107341642936117619902_e1ed52697ebc2587'

# Alternativamente, use:
# os.environ['MARITALK_API_SECRET_KEY'] = 'sua-chave-aqui'

# Para OpenAI (se necessário):
# os.environ['OPENAI_API_SECRET_KEY'] = 'sk-sua-chave-aqui'

print("✅ Chaves API configuradas")
print(f"   Maritaca: {'✅' if os.environ.get('CURSORMINIMAC') or os.environ.get('MARITALK_API_SECRET_KEY') else '❌'}")
print(f"   OpenAI: {'✅' if os.environ.get('OPENAI_API_SECRET_KEY') else '❌ (opcional)'}")


In [ ]:
# Testar conexão com API Maritaca
import openai

api_key = os.environ.get('CURSORMINIMAC') or os.environ.get('MARITALK_API_SECRET_KEY')

if api_key:
    openai.api_base = "https://chat.maritaca.ai/api"
    
    # Detectar versão do openai
    openai_version = openai.__version__
    major_version = int(openai_version.split('.')[0])
    
    if major_version >= 1:
        client = openai.OpenAI(api_key=api_key, base_url="https://chat.maritaca.ai/api")
        response = client.chat.completions.create(
            model="sabia-3",
            messages=[{"role": "user", "content": "Responda apenas: OK"}],
            max_tokens=5
        )
        print(f"✅ API funcionando! Resposta: {response.choices[0].message.content}")
    else:
        openai.api_key = api_key
        response = openai.ChatCompletion.create(
            model="sabia-3",
            messages=[{"role": "user", "content": "Responda apenas: OK"}],
            max_tokens=5
        )
        print(f"✅ API funcionando! Resposta: {response.choices[0].message.content}")
else:
    print("❌ Chave API não configurada")


## 4️⃣ Verificar Dados ENEM


In [ ]:
import json
from pathlib import Path

# Verificar se os dados estão presentes
data_dir = Path('data/enem')

if data_dir.exists():
    print("✅ Diretório de dados encontrado")
    for file in data_dir.glob('*.jsonl'):
        count = sum(1 for _ in open(file))
        print(f"   {file.name}: {count} questões")
else:
    print("⚠️  Diretório de dados não encontrado")
    print("   Os dados serão baixados automaticamente na primeira execução")


## 5️⃣ Executar Avaliação

Agora você pode executar avaliações do ENEM!


In [ ]:
# Exemplo: Avaliar Sabiá-3 no ENEM 2024 (teste rápido com --limit)
!python main.py \
    --model maritalk \
    --model_args engine=sabia-3 \
    --tasks enem_cot_2024_blind \
    --description_dict_path description.json \
    --num_fewshot 3 \
    --conversation_template chatgpt \
    --limit 5 \
    --output_path results/teste_rapido.json


In [ ]:
# Avaliação completa (sem --limit)
# ⚠️  Isso pode levar muito tempo e consumir muitos créditos da API

# !python main.py \
#     --model maritalk \
#     --model_args engine=sabia-3 \
#     --tasks enem_cot_2024_blind \
#     --description_dict_path description.json \
#     --num_fewshot 3 \
#     --conversation_template chatgpt \
#     --output_path results/sabia3_enem2024_completo.json


## 6️⃣ Analisar Resultados


In [ ]:
# Carregar e visualizar resultados
import json
from pathlib import Path

# Criar diretório de resultados se não existir
Path('results').mkdir(exist_ok=True)

# Carregar resultados (ajuste o caminho conforme necessário)
result_file = 'results/teste_rapido.json'

if Path(result_file).exists():
    with open(result_file, 'r', encoding='utf-8') as f:
        results = json.load(f)
    
    print("📊 Resultados da Avaliação:")
    print("=" * 70)
    
    if 'results' in results:
        for task_name, task_results in results['results'].items():
            print(f"\n📋 Tarefa: {task_name}")
            if 'acc' in task_results:
                print(f"   ✅ Acurácia geral: {task_results['acc']:.2%}")
            
            # Áreas de conhecimento
            areas = {
                'languages': 'Linguagens e Códigos',
                'human-sciences': 'Ciências Humanas',
                'natural-sciences': 'Ciências da Natureza',
                'mathematics': 'Matemática'
            }
            
            for area_key, area_name in areas.items():
                if area_key in task_results:
                    acuracia = task_results[area_key]
                    barra = "█" * int(acuracia * 30)
                    print(f"   • {area_name:25s}: {acuracia:6.2%} {barra}")
    else:
        print(json.dumps(results, indent=2, ensure_ascii=False))
else:
    print(f"⚠️  Arquivo não encontrado: {result_file}")
    print("   Execute uma avaliação primeiro!")


In [ ]:
# Visualização gráfica dos resultados (opcional)
try:
    import matplotlib.pyplot as plt
    import numpy as np
    
    result_file = 'results/teste_rapido.json'
    
    if Path(result_file).exists():
        with open(result_file, 'r') as f:
            results = json.load(f)
        
        if 'results' in results:
            for task_name, task_results in results['results'].items():
                areas = {
                    'languages': 'Linguagens',
                    'human-sciences': 'Humanas',
                    'natural-sciences': 'Natureza',
                    'mathematics': 'Matemática'
                }
                
                area_names = []
                acuracias = []
                
                for area_key, area_name in areas.items():
                    if area_key in task_results:
                        area_names.append(area_name)
                        acuracias.append(task_results[area_key])
                
                if area_names:
                    plt.figure(figsize=(10, 6))
                    bars = plt.bar(area_names, acuracias, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
                    plt.title(f'Acurácia por Área - {task_name}', fontsize=14, fontweight='bold')
                    plt.ylabel('Acurácia (%)', fontsize=12)
                    plt.ylim(0, 1)
                    plt.grid(axis='y', alpha=0.3)
                    
                    # Adicionar valores nas barras
                    for bar, acc in zip(bars, acuracias):
                        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                               f'{acc:.1%}', ha='center', va='bottom', fontweight='bold')
                    
                    plt.tight_layout()
                    plt.show()
except ImportError:
    print("💡 Instale matplotlib para visualizações: !pip install matplotlib")
except Exception as e:
    print(f"⚠️  Erro na visualização: {e}")


## 7️⃣ Download dos Resultados

Para baixar os resultados, use:
- **Files → Download** (clique com botão direito no arquivo)
- Ou execute a célula abaixo


In [ ]:
from google.colab import files

# Listar arquivos de resultados disponíveis
result_files = list(Path('results').glob('*.json'))

if result_files:
    print("📁 Arquivos de resultados disponíveis:")
    for i, file in enumerate(result_files, 1):
        size = file.stat().st_size / 1024  # KB
        print(f"   {i}. {file.name} ({size:.1f} KB)")
    
    print("\n💡 Para fazer download:")
    print("   1. Vá em Files (ícone de pasta à esquerda)")
    print("   2. Navegue até results/")
    print("   3. Clique com botão direito → Download")
    print("\n   Ou descomente a linha abaixo para download automático:")
    print("   # files.download('results/teste_rapido.json')")
else:
    print("⚠️  Nenhum arquivo de resultados encontrado")
    print("   Execute uma avaliação primeiro!")


## 💡 Dicas e Observações

### GPU
- **Gratuito**: T4 (15GB RAM)
- **Colab Pro**: T4, V100, A100 (40GB RAM)
- **Colab Pro+**: A100 prioritário

### Limites
- Sessões gratuitas: ~12 horas
- Colab Pro: ~24 horas
- Use `--limit` para testes rápidos

### Custos API
- Monitore uso da API Maritaca
- Avaliação completa pode ser cara
- Use cache quando possível (`--no_cache` desabilitado)

### Performance
- API Maritaca não usa GPU local (é API remota)
- GPU é útil para modelos locais (transformers)
- Para API, GPU não é necessária, mas Colab oferece ambiente estável

### Backup
- ⚠️ **IMPORTANTE**: Faça download dos resultados regularmente
- Dados são temporários e serão perdidos ao desconectar
- Use `--output_path` para salvar resultados


## 🔄 Exemplos de Uso Avançado

### Múltiplas Tarefas


In [ ]:
# Avaliar múltiplas tarefas de uma vez
# !python main.py \
#     --model maritalk \
#     --model_args engine=sabia-3 \
#     --tasks enem_cot_2024_blind,enem_cot_2024_captions \
#     --description_dict_path description.json \
#     --num_fewshot 3 \
#     --conversation_template chatgpt \
#     --output_path results/multiplas_tarefas.json


### Comparar Diferentes Configurações


In [ ]:
# Comparar diferentes números de few-shot
# for num_fewshot in [0, 1, 3]:
#     !python main.py \
#         --model maritalk \
#         --model_args engine=sabia-3 \
#         --tasks enem_cot_2024_blind \
#         --description_dict_path description.json \
#         --num_fewshot {num_fewshot} \
#         --conversation_template chatgpt \
#         --limit 10 \
#         --output_path results/fewshot_{num_fewshot}.json


### Verificar Status da Sessão


In [ ]:
# Verificar informações da sessão
import psutil
import os
from datetime import datetime

print("📊 Status da Sessão Colab")
print("=" * 50)
print(f"🕐 Hora atual: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💾 RAM disponível: {psutil.virtual_memory().available / 1e9:.2f} GB")
print(f"💾 RAM total: {psutil.virtual_memory().total / 1e9:.2f} GB")
print(f"📁 Diretório atual: {os.getcwd()}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memória GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"💾 GPU livre: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
else:
    print("⚠️  GPU não disponível")
